###### I tried to summerize my understanding of the following paper in this notebook. 

---
Title: Multiple receptive fields and small-object-focusing weakly-supervised segmentation network for fast object detection.

Authors: Siyang Sun and Yingjie Yin and Xingang Wang and De Xu and Yuan Zhao and Haifeng Shen

Link: https://arxiv.org/abs/1904.12619

---

### Motivation

Precision and speed of current SOTA models are contradictory. We have to sacrifice one over another. In general, CNN models for object detection can be divided into two classes-

   1. Region based two stage detector: Two-stage detectors achieve higher precision, however, their complex computation and lower speed limited the practical application.
    
   2. Region free one stage detector: running speed keep real time performance, but the accuracy has a clear drop which is about 10% to 40% relative to state-of-the-art two- stage detector. The reason for this low precision is, this type of models can not detect small objects correctly.
    
In this paper, they tried to achieve good precision score while keeping the speed. They proposed MRFSWSnet and with that they managed to achieve 80.9% mAP score on Pascal VOC test dataset. 

### Contribution

Their contribution can be divided into two main category. 

   1. Multiple Receptive Fileds (MRF) block: It improves detection accuracy. It also Enhance the features discriminability by focusing on objects and its adjacent background with different weights.
   
   2. Weakly supervised segmentation module: This module only focuses on small objects, instead of all objects to improve precision of small object detection. It uses two thresholds to define a small objects. 

### Network Architecture

Following is the network architecture of MRFSWSnet:

<img src="images/MRFSWSnet_arch.png"/>

As we can see the network is composed of two main branches. 
   1. detection branch: It reuses the structure of SSD which means base network is VGG16 trained on ImageNet. They add one extra feature layer called conv3x3E from the feature pyramid network(FPN). With a rough feature of conv9 they upsample the feature and merge it with the bottom-up feature. They repeat the process until conv3x3 to get conv3x3_E. Each features of SSD except for conv8 and conv9, follows by one MRF block. MRF block can not be applied to these two layers because of low resolution of these two layers 1x1 and 3x3 respectively. 
   
   2. segmentation branch: This branch focuses only on small objects to improve the precision. We will see this branch in details in later part. 
   
The detection branch and segmentation branch are combined in the form of multiple tasks. 

### MRF block

<table>
       <tr>
            <th><img src="images/MRF_block.png" width="300"/></th>
            <th><img src="images/MRF_block1.png" width="300"/></th>
       </tr>
</table>

MRF block composed of multiple convolution layers with different receptive field. First part of MRF is similar with Inception structure, whose components are several convolution layers with different kernel sizes, including 1×1 convolution layer, 3×3 convolution layer and 5×5 convolution layer. The receptive fields of these convolution layers can cover more positions of one object from its center to its surrounding. 

The other part of MRF includes several dilated convolution layers with different dilated ratios, such as 3×3 convolution layer with dilated ratios 1, 2 and 3, respectively. These convolution layers pay attention to different spatial locations with different weights according to different distances from objects’ center position to its surrounding.

It also use shortcut connections like resnet to maintain the performance of previous layer. They mentioned two advantages of this kind of block: 

   1. It gives more importance to the center pixels of the object.
   2. more contextual information is also focused by larger receptive fields.

### Small object focusing weakly supervised segmentation (SWS) block 

<table>
       <tr>
            <th><img src="images/SWS_module.png" width="300"/></th>
            <th><img src="images/sws_module1.png" width="300"/></th>
       </tr>
</table>

The input to this module is Conv3_3E which is obtained from FPN. Ground truth for SWS is generated by only focusing on small object's bounding box without considering the class of the object. Thats why it is called weakly supervised. They use two thresholds T1 and T2 for selecting the small objects. 
   1. The objects whose areas are in [T1, T2] are labeled as foreground. 
   2. If the area is greater than T2 that object is considered as background. 
   3. If the area is smaller than T1, then ignore that object as well. cause extreme small object has not enough effective information for training segmentation module.

The module consists of two transposed conv layer and one transition conv layer with relu non-linearity following conv3_3E. Finally, a binary softmax classifier is designed to predict probability score of each pixel in the output mask as shown in the figure above. 

### Training

The final loss function includes the binary cross-entropy loss for semantic segmentation, the softmax loss for classification and the smooth-l1 loss function for localization. Detection loss is same as SSD loss. 

The training process is little different from original SSD. At the beginning of training, the learning rate has a “warm up” process, whose range is from 10^-6 to 10^-4 at the first 10 epochs. After 10 epochs, the learning rate
is set to 10^-4 until 150-th epochs. Then, it is divided by 10 at 150-th epoch and 250-th. The total number of training epochs is set to 300. The weight decay is 0.0005, the momentum is 0.9, and the batch size is set to 32.

T1 and T2 are set as 1024 and 9216. We can choose T1 and T2 wisely if we have a good idea about our dataset. 

### Results

They trained their model on PASCAL VOC and MS COCO both. Their results are summerized in these tables: 

<table>
       <tr>
            <th><img src="images/pascal_voc_res.png" width="300"/></th>
            <th><img src="images/ms_coco_res.png" width="300"/></th>
       </tr>
</table>

If you consider both time and speed, I think this model is doing well. 

### Ablation experiments

They also did ablation study and try to find out how much each component contribute to the precision. As we can see every component increase the precision a little bit. 

<table>
       <tr>
            <th><img src="images/ablation.png" width="300"/></th>
            <th><img src="images/performance.png" width="300"/></th>
       </tr>
</table>

### Conclusion

Two main contribution of this paper was-
   1. MRF block
   2. Weakly supervised segmentation module for small object. 


In the end this paper is very easy to understand and very simple to implement. 